In [1]:
import csv
import gzip
import os

#Takes in two strings, one from the format column and one from the column that 
#specifies the data. Example inputs: 'GT:GQX:DP:DPF' as format_string, and '0/0:21:8:0'
#as data_string. Splits these strings and returns a string that represents the genotype
#enclosed in brackets. 
def get_genotype(format_string, data_string):
    format_list = format_string.split(":")
    data_list = data_string.split(":")
    info = {} #a dict of format subtype string --> data subtype string
    for f, d in zip(format_list, data_list):
        info[f] = d
    return '[' + info['GT'] + ']'

In [ ]:
with gzip.open('/40TB_3/InovaGenomes/RCF/NORM/20150920/LP6005636-DNA_F04.norm.sorted.rcf.gz') as rcfFile:
    pos_category = []
    chrom = 'chr1' 
    rcfReader = csv.reader(rcfFile, delimiter = '\t')
    col_labels = ['CHROM', 'START','END', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'Sample']
    indexes = range(0, len(col_labels))
    col_dict= {}
    for index, label in zip(indexes, col_labels):
         col_dict[label] = index
    for i, row in enumerate(rcfReader):
        if(not(row[0].startswith('#'))):
            current_chrom = row[col_dict['CHROM']]
            pos = row[col_dict['START']]
            category = row[col_dict['FILTER']]
            if(category == 'PASS'):
                formatD = row[col_dict['FORMAT']]
                data = row[col_dict['Sample']]
                category += get_genotype(formatD, data)
            if(current_chrom != chrom): #if we have reached a new chromsome, create a file for the old
                #create file for the old chrom
                with open('%s position and category.rcf'%chrom, 'wb') as output_file:
                    output_writer = csv.writer(output_file, delimiter = '\t')
                    for sublist in pos_category:
                        output_writer.writerow(sublist) #print each line's position and category
                #reset variables for the new chromosome
                del pos_category[:]
                chrom = current_chrom
            
            #regardless of whether chrom changed, a new entry will be added to the map
            pos_category.append([pos, category]) #how about duplicate postions?
        if i >= 183: break

    #When we have reached the end of the file, the last chromosome's data would not have been written yet. 
    #The way my code is set up, it doesn't print the file until a new chromsome appears. 
    #Writing file for last chromosome below. 
    with open('%s position and category.rcf'%chrom, 'wb') as output_file:
                    output_writer = csv.writer(output_file, delimiter = '\t')
                    for sublist in pos_category:
                        output_writer.writerow(sublist)